In [1]:
import numpy as np
from stl import mesh
from PIL import Image

np.set_printoptions(precision=3, suppress=True)

#import and adjust image
source_img = Image.open('smiley.png').convert("L")
img = np.asarray(source_img) / 255.0 # normalized grayscale
gray_range = img.max() - img.min() #find range of values
img = (img - img.min()) / (gray_range) #stretch contrast 0.0 to 1.0
nw, nh = img.shape #store pixel size of source image

<frozen importlib._bootstrap>:488: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.


In [2]:
#define square tile centers
W = 20
H = 20
g = 1 # width of transition triangles ("grout")
wd = 5+g # width of each tile area
hd = 5+g # height of each tile area
wr = wd/2 # half width of mirror spacing
hr = hd/2 # half width of mirror spacing
#centers are d apart, starting at r. first corner is at 0,0
Xctr_v = np.array(range(W))*wd + wr
Yctr_v = np.array(range(H))*hd + hr
#create a grid of coordinate pairs from these vectors
Xctr_a, Yctr_a = np.meshgrid(Xctr_v,Yctr_v)
ctr_a = np.dstack((Xctr_a, Yctr_a))

In [3]:
# map grid x position to heading direction
x_range = ctr_a[-1,-1,0]-ctr_a[0,0,0] # find range of x ctr coordinates
hdg_a = 2*(ctr_a[:,:,0]-ctr_a[0,0,0])/x_range - 1.0 # create normalized heading array
shp = .5 # how much of the curved tails of the rolloff shape to capture
pan = np.pi/15 # -12 to +12 degrees
hdg_a = np.tanh(shp*hdg_a)/np.tanh(shp) * np.pi/12 # use tanh rolloff
# add a little random noise to headings
np.random.seed(42)
noise_strength = np.pi/90 # 2 degree stdev of the noise
wiggler = np.random.default_rng() # create random number generator
hdg_a = hdg_a + wiggler.normal(0.0, noise_strength, size=(H, W)) # add heading noise

In [4]:
#subsample the image under each tile and average the pixel values

elv_a = hdg_a*0.0 #initialize the elevation array
#find the total height of the mirror array
ah = ctr_a[-1,0,1] + hr
#find the total width of the mirror array
aw = ctr_a[0,-1,0] + wr
#scale each tile to the image pixel space (half tile size)
sw = int(round(wr*nw/aw,0))
sh = int(round(hr*nh/ah,0))
#step through tile center locations, using y position matrix as iterator
for i, row in enumerate(ctr_a[:,:,1]):
    for j, y in enumerate(row):
        x = ctr_a[i,j,0] # pull matching x coordinate
        Y = int(round(y*(nh-.5)/ah,0)) # translate this coord to image space
        X = int(round(x*(nw-.5)/aw,0))
        # slice cropped copy of the image in this tile's region
        temp = [X-sw, X+sw, Y-sh, Y+sh]
        temp[0] = max(temp[0],0)
        temp[1] = min(temp[1],nw)
        temp[2] = max(temp[2],0)
        temp[3] = min(temp[3],nh)
        # sample the mean brightness in this slice as the elevation (inverted from before)
        if sw==0:
            if sh==0:
                elv_a[i,j] = img[temp[0],temp[2]]
            else:
                elv_a[i,j] = img[temp[0],temp[2]:temp[3]].mean()
        else:
            if sh==0:
                elv_a[i,j] = img[temp[0]:temp[1],temp[2]].mean()
            else:
                elv_a[i,j] = img[temp[0]:temp[1],temp[2]:temp[3]].mean(axis=0).mean()

elv_a = 1.0 - elv_a #invert elevation because low angles are brighter at sunset/sunrise
#grayscale to elevation angle
gamma = 1.5
min_elv = -np.pi/30 # -6 degrees min elevation
max_elv =  np.pi/9  # 20 degrees max elevation
elv_a = (elv_a ** gamma) * (max_elv - min_elv) + min_elv

In [5]:
# generate generic corner points, as rows in an array
th = np.pi/3 # 60 degrees
pt0_a = [[wr-g/2,hr-g/2,0],[-wr+g/2,hr-g/2,0],[-wr+g/2,-hr+g/2,0],[wr-g/2,-hr+g/2,0]]

# generate a larger array of scaled, rotated, and positioned corner points
tile_a = np.empty((H,W,4,3))
for i, row in enumerate(ctr_a[:,:,0]): # use x for iterator
    for j, x in enumerate(row):
        y = ctr_a[i,j,1] # pull matching y coord
        hd = hdg_a[i,j] # pull this heading angle
        el = elv_a[i,j] # pull this elevation angle
        #create two rotation matrices
        r3h = np.array([[np.cos(hd),0,-np.sin(hd)],[0,1,0],[np.sin(hd),0,np.cos(hd)]])
        r3e = np.array([[1,0,0],[0,np.cos(el),-np.sin(el)],[0,np.sin(el),np.cos(el)]])
        #add these rotated and translated corner points to the list of tiles
        tile_a[i,j,:,:] = pt0_a@r3e@r3h + [x,y,0]

In [6]:
# create a vector of bottom edge points
be_v = ctr_a[0,:,:]-[wr,0] # copy lower row centers and shift half a tile left
be_v[:,1] = -hr/2 # lower the y coordinates to make the border
be_v[0,0] = -wr/2 # extend left corner a bit to match up to the border
be_v = np.vstack((be_v,[aw+wr/2,-hr/2])) # add an additional right corner point
be_v = np.hstack((be_v,np.zeros((W+1,1)))) # add z coordinates of 0

# create a vector of left edge points similarly
le_v = ctr_a[:,0,:]-[0,hr]
le_v[:,0] = -wr/2
le_v[0,1] = -hr/2
le_v = np.vstack((le_v,[-wr/2,ah+hr/2]))
le_v = np.hstack((le_v,np.zeros((H+1,1))))

#create a vector of top edge points
te_v = be_v + [0,ah+hr,0]

#create a vector of right edge points
re_v = le_v + [aw+wr,0,0]

#shift most of bottom edge forward to avoid tipping forward
be_v[1:-1,2] = hr


In [7]:
tri_a = []
#do some edge flipping optimization for "quads"
def quad_add(q):
    if np.linalg.norm(q[0] - q[2]) < np.linalg.norm(q[1] - q[3]):
        tri_a.append([q[0],q[1],q[2]])
        tri_a.append([q[0],q[2],q[3]])
    else:
        tri_a.append([q[0],q[1],q[3]])
        tri_a.append([q[1],q[2],q[3]])
        
#create main mirror face
for i, row in enumerate(tile_a):
    for j, FA in enumerate(row):
        
        if i<(H-1):
            UP = tile_a[i+1,j] # the Face Array above this one
        else:
            UP = FA*0.0
        if j<(W-1):
            RT = tile_a[i,j+1] # the Face Array to the right of this one
        else:
            RT = FA*0.0
        
        # main face
        quad_add((FA[0],FA[1],FA[2],FA[3]))
        
        if i==0: # bottom skirt
            quad_add(( FA[3], FA[2], be_v[j], be_v[j+1] ))
        # grout on top of each row
        if i<(H-1):
            quad_add((FA[1], FA[0], UP[3], UP[2]))
            if j==0: # add interstitial triangles for left skirt
                tri_a.append([ FA[1], UP[2], le_v[i+1] ])
            elif j==(W-1): # add interstitial triangles for right skirt
                tri_a.append([ FA[0], re_v[i+1], UP[3] ])
        # top skirt
        else:
            quad_add((FA[1], FA[0], te_v[j+1], te_v[j]))
        
        if j==0: # left skirt
            quad_add((FA[2], FA[1], le_v[i+1], le_v[i]))
        # grout on right side of each column:
        if j<(W-1):
            quad_add((FA[0], FA[3], RT[2], RT[1]))
            if i==0: # add interstitial triangles for bottom skirt
                tri_a.append([ FA[3], be_v[j+1], RT[2] ])
            elif i==(H-1): # add interstitial triangles for top skirt
                tri_a.append([ FA[0], RT[1], te_v[j+1] ])
        else: # right skirt
            quad_add((FA[0], FA[3], re_v[i], re_v[i+1]))
            
        if j<(W-1) and i<(H-1): # add triangles in top right corner
            UPRT = tile_a[i+1,j+1]
            quad_add((FA[0], RT[1], UPRT[2], UP[3]))

In [8]:
# create back face
thk = ah/6
C = [aw/2, ah/2, -thk/3]
slope = [[0,0,(2-d/(len(le_v)-1))*thk/2] for d in range(len(le_v))]
vbak = np.array([[0,0,3*thk/2-((1-2*d/(len(be_v)-1))**2)*thk/2] for d in range(len(be_v))])
vbak[1:-1,2] = vbak[1:-1,2]+hr
for i in range(len(be_v)-1):
    tri_a.append([ C, be_v[i+1]-vbak[i+1], be_v[i]-vbak[i] ])
    tri_a.append([ C, te_v[i]-[0,0,thk/2], te_v[i+1]-[0,0,thk/2] ])
for i in range(len(le_v)-1):
    tri_a.append([ C, le_v[i]-slope[i], le_v[i+1]-slope[i+1] ])
    tri_a.append([ C, re_v[i+1]-slope[i+1], re_v[i]-slope[i] ])

In [9]:
# create four sides
# bottom
t1 = np.array([ be_v[:-1], be_v[:-1]-vbak[:-1], be_v[1:] ]).transpose(1,0,2)
t2 = np.array([ be_v[1:], be_v[:-1]-vbak[:-1], be_v[1:]-vbak[1:] ]).transpose(1,0,2)
# top
t3 = np.array([ te_v[:-1], te_v[1:], te_v[:-1]-[0,0,thk/2] ]).transpose(1,0,2)
t4 = np.array([ te_v[1:], te_v[1:]-[0,0,thk/2], te_v[:-1]-[0,0,thk/2] ]).transpose(1,0,2)
# ...and stack
tri_a = np.vstack((tri_a,t1,t2,t3,t4))
# left
t1 = np.array([ le_v[:-1], le_v[1:], le_v[:-1]-slope[:-1] ]).transpose(1,0,2)
t2 = np.array([ le_v[1:], le_v[1:]-slope[1:], le_v[:-1]-slope[:-1] ]).transpose(1,0,2)
# right
t3 = np.array([ re_v[:-1], re_v[:-1]-slope[:-1], re_v[1:] ]).transpose(1,0,2)
t4 = np.array([ re_v[1:], re_v[:-1]-slope[:-1], re_v[1:]-slope[1:] ]).transpose(1,0,2)
# ...and stack
tri_a = np.vstack((tri_a,t1,t2,t3,t4))

In [10]:
# convert to numpy array
faces_array = np.asarray(tri_a)

# Create mesh
surf_mesh = mesh.Mesh(np.zeros(faces_array.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces_array):
    surf_mesh.vectors[i] = f

# Export STL
surf_mesh.save('grid_mirror.stl')